<a href="https://colab.research.google.com/github/sfmajors373/FourthBrainBreastCancer/blob/seg_UNet_update/V2_FEATURE_EXTRACTOR_train_model_inception_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This version use a different data loading method to load tiles 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import os
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import random
import h5py
import math
import pickle
from datetime import datetime
from numba import cuda 

%matplotlib inline

In [ ]:
# Code snippet nvidia gpu
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
HDF5_FOLDER = '/content/drive/MyDrive/Colab Notebooks/CancerMap/train'
green_layer_only = False
color_normalization_json="CAMELYON16_color_normalization.json"

In [ ]:
# Mannual Data Loading Methods
import h5py
import os

cancer_image = []
cancer_label = []
cancer_mask = []

file = os.listdir(HDF5_FOLDER)
for f_index in range(0, len(file)):
  if file[f_index].find('.hdf5') != -1:
    filepath = HDF5_FOLDER+'/'+file[f_index]

    f = h5py.File(filepath, 'r')
    f_name = list(f.keys())
    dset = np.array(f[f_name[0]])
    dset=dset[:,0:256, 0:256, :]
    print(dset.shape)
    if file[f_index].find('mask') != -1:
      cancer_mask.append(dset)
    elif file[f_index].find('normal') != -1:
      # cancer_label.append(0)
      # cancer_image.append(dset)
      labels = [0]*len(dset)

      if f_index == 0:
        cancer_image = dset
        cancer_label = labels
      else:
        cancer_image= np.concatenate((cancer_image, dset), axis = 0)
        cancer_label = np.concatenate((cancer_label, labels), axis = 0)
    else:
      labels = [1]*len(dset)
      if f_index == 0:
        cancer_image = dset
        cancer_label = labels
      else:
        cancer_image= np.concatenate((cancer_image, dset), axis = 0)
        cancer_label = np.concatenate((cancer_label, labels), axis = 0)

# print(cancer_image)
# print(cancer_label)
# print(cancer_mask)

In [ ]:
print(len(cancer_image))
print(len(cancer_label))



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cancer_image, cancer_label, test_size = 0.2, random_state = 42)

In [ ]:
print(len(X_train))
print(len(X_test))

In [ ]:
# Plot model loss and accuracy
  
def plot_history(history):
  # This function will plot the model fit process
  print(history.history.keys())
  # summarize history for accuracy
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('acc')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()


In [ ]:
# FEATURE EXTRACTOR MODEL

In [ ]:
model = keras.applications.InceptionV3(input_shape=(256,256,3),
                                        include_top=True,
                                        classes = 1,
                                        classifier_activation="sigmoid",
                                        weights=None)


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=10, epochs=40)

In [ ]:
plot_history(history)

## Display layer 0 of activation of a normal image

In [ ]:
from keras.models import Model
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
image_num = 11600 #normal
activation = activation_model.predict(X_train[image_num].reshape(1,256,256,3))





In [ ]:
y_train[image_num]

In [ ]:
len(activation)

In [ ]:
layer_act = 0

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 0])

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 1])

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 2])

## Display layer 0 of activation of a tumor image

In [ ]:
image_num = 1
activation = activation_model.predict(X_train[image_num].reshape(1,256,256,3))

In [ ]:
y_train[image_num]

In [ ]:
len(activation)

In [ ]:
layer_act = 0

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 0])

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 1])

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 2])

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/CancerMap/model_inceptionv3.hdf5')

In [ ]:
# Test Accuracy
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score,precision_score,recall_score
prediction_values = np.argmax(model.predict(X_test), axis=-1)
print(f'Accuracy = {accuracy_score(y_test, prediction_values)}')
print(f'F1 = {f1_score(y_test, prediction_values,average = None)}')
print(f'Precision = {precision_score(y_test, prediction_values,average=None)}')
print(f'Recall = {recall_score(y_test, prediction_values,average = None)}')
# Display the confusion matrix
print('Confusion matrix =')
print(confusion_matrix(y_test, prediction_values))

In [ ]:
feature_network = Model(model.input, model.get_layer('avg_pool').output)
feature_train = feature_network.predict(X_train)
# feature_test = feature_network.predict(test_data_array)

In [ ]:
np.savetxt("/content/drive/MyDrive/Colab Notebooks/CancerMap/feature_extracted.txt", feature_train)